### Supplementary Figure 6 | Treatment status modulates pathway-level responses to vaccination in sarcoidosis
Stratified pathway analyses compare transcriptional responses between treated and untreated sarcoidosis participants.